In [1]:
import numpy as np
import pandas as pd

In [4]:
pwd

'/Users/priyankashrestha/Documents/stanford/courses_senior/cs229/cs229_final_project'

In [7]:
pd.read_csv('../cs229_final_project_data/project_data/chief_complaint.csv')

,Unnamed: 0,0,1
0,0,Worsening ABD distension and pain Major Su...,F
1,1,abdominal fullness and discomfort Major Sur...,F
2,2,altered mental status Major Surgical or Inv...,F
3,3,Abdominal pain Major Surgical or Invasive P...,F
4,4,Visual hallucinations Major Surgical or Inv...,M
...,...,...,...
331788,331788,Enterocutaneous/enteroatmospheric fistula M...,F
331789,331789,NaN,F
331790,331790,seizure Major Surgical or Invasive Procedur...,M
331791,331791,seizures Major Surgical or Invasive Procedu...,M


In [3]:
discahrge = pd.read_csv('./raw_data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/discharge.csv.gz', compresion="gzip")

In [8]:
discahrge['text'][0]

' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eatin